# ♟️ Global Chess Challenge 2025: Qwen2.5-7B Training

This notebook trains a **7B parameter model** (Qwen 2.5) on the ChessExplained dataset.

### 🚀 Setup
1. **GPU**: Use 2x T4 or A100.
2. **Internet**: Enabled.
3. **Secrets**: Add `WANDB_API_KEY` (optional).

In [ ]:
# 1. Install Dependencies & Repository
!pip install datasets transformers accelerate chess wandb jinja2 vllm
!pip install git+https://github.com/AIcrowd/chess-env.git
!apt-get update && apt-get install -y stockfish

# Install Baselines Repo directly (This provides callbacks, helpers, etc.)
!pip install git+https://github.com/ritwikareddykancharla/Global-Chess-Challenge-2025-Baselines.git

In [ ]:
# 2. prepare Data & Tokenizer (Dynamic Setup)
from transformers import AutoTokenizer
from datasets import load_dataset
import chess
import os

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

# A. Load Dataset from Hugging Face
print("Loading dataset from Hugging Face...")
dataset = load_dataset(
    "parquet", 
    data_files="https://huggingface.co/datasets/aicrowd/ChessExplained/resolve/main/ChessExplained_2500k_qwen3.parquet",
    split="train"
)

# B. Build Custom Tokenizer for 7B Model
print("Building tokenizer with special chess tokens...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Add Special Tokens
squares = [f"<{chess.square_name(sq)}>" for sq in chess.SQUARES]
pieces = [
    '<White_Pawn>', '<White_Knight>', '<White_Bishop>', 
    '<White_Rook>', '<White_Queen>', '<White_King>',
    '<Black_Pawn>', '<Black_Knight>', '<Black_Bishop>',
    '<Black_Rook>', '<Black_Queen>', '<Black_King>',
    '<blank>'
]
tokenizer.add_special_tokens({'additional_special_tokens': squares + pieces})
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.save_pretrained("chess_tokenizer_7b")
TOKENIZER_PATH = "chess_tokenizer_7b"

In [ ]:
# 3. Training Loop
import torch
from datasets import Dataset
import wandb
from transformers import (
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
# Import directly from installed package
from chess_evaluation_callback import ChessLLMEvaluationCallback

# Configuration
OUTPUT_DIR = "./trained_models/qwen7b_chess"
NUM_LINES_TO_LOAD = 500_000 # Start small, increase for full run
MAX_LENGTH = 512
EVAL_STEPS = 2000

wandb.init(project="ChessLLM", name="qwen2.5-7b-kaggle")

# Load Model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2",
)
model.resize_token_embeddings(len(tokenizer))

# Load Data (Already loaded in Step 2)
dataset = dataset.select(range(NUM_LINES_TO_LOAD))
tokenized_dataset = dataset.map(lambda x: tokenizer(x["text"], truncation=True, max_length=MAX_LENGTH), batched=True)

# Trainer
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-5, # Low LR for 7B
    bf16=True,
    logging_steps=100,
    save_steps=EVAL_STEPS,
    report_to="wandb",
    num_train_epochs=1
)

evaluation_callback = ChessLLMEvaluationCallback(
    model=model,
    tokenizer=tokenizer,
    checkpoint_dir=OUTPUT_DIR,
    eval_every_n_steps=EVAL_STEPS,
    output_dir="./eval_results",
    batch_size=4,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[evaluation_callback]
)

trainer.train()
trainer.save_model(f"{OUTPUT_DIR}/final")
tokenizer.save_pretrained(f"{OUTPUT_DIR}/final")